In [125]:
import torch
import time
import re
import json
import pandas as pd
from datasets import Dataset
from trl import GRPOConfig, GRPOTrainer, apply_chat_template
from peft import LoraConfig, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer
# from math_verify import LatexExtractionConfig, parse, verify

In [126]:
def create_prompt(train_sample):
    SYSTEM_PROMPT = (
    "You are playing the NY Times Connections game. I will give you a set of 16 words, and I want you to provide 4 sets of exactly 4 words that are connected in some way. \
    I want you to group the words in such a way that each group has a common theme. Think about your answers carefully, as you will only have one chance to submit your answer. \
    Here is an example: If the words are: 'BUCKS, HAIL, JAZZ, SHIFT, LEVEL, MOM, SNOW, RACECAR, SLEET, TAB, KAYAK, RETURN, OPTION, NETS, RAIN, HEAT', \
    a possible answer could be: 'answer: [['HAIL', 'RAIN', 'SLEET', 'SNOW'], ['BUCKS', 'HEAT', 'JAZZ', 'NETS'], ['OPTION', 'RETURN', 'SHIFT', 'TAB'], ['KAYAK', 'LEVEL', 'MOM', 'RACECAR']] and groups: ['WET WEATHER', 'NBA TEAMS', 'KEYBOARD KEYS', 'PALINDROMES']. \
    Give your answer strictly in the format (no other words): \
            'Answer: [[4 words of group1], [4 words of group2], [4 words of group3], [4 words of group4]]  \
            Group: [group1, group2, group3, group4].'\
    "
    )
    
    return {
        "prompt": [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": train_sample["question"]},
        ],
    }

In [127]:
df = pd.read_csv("../data/connections.csv", index_col=None)
train_dataset = df.iloc[:500]
test_dataset = df.iloc[500:]

train_dataset["prompt"] = train_dataset.apply(create_prompt, axis=1)
test_dataset["prompt"] = test_dataset.apply(create_prompt, axis=1)

/var/folders/t8/syttdns55xbgn_9wnxgg6gh00000gn/T/ipykernel_89215/2853767823.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_dataset["prompt"] = train_dataset.apply(create_prompt, axis=1)
/var/folders/t8/syttdns55xbgn_9wnxgg6gh00000gn/T/ipykernel_89215/2853767823.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset["prompt"] = test_dataset.apply(create_prompt, axis=1)


In [128]:
# class ConnectionsDataset(Dataset):
#     def __init__(self, dataframe, tokenizer, max_length=128):
#         self.dataframe = dataframe.reset_index(drop=True)
#         self.tokenizer = tokenizer
#         self.max_length = max_length

#     def __len__(self):
#         return len(self.dataframe)

#     def __getitem__(self, idx):
#         prompt_data = self.dataframe.iloc[idx]['prompt']
#         answer = self.dataframe.iloc[idx]['answers']

#         prompt_data = json.loads(prompt_data)

#         prompt = apply_chat_template(
#                 {"prompt": prompt_data},
#                 self.tokenizer,
#                 tokenize=False,
#                 add_generation_prompt=True
#             )
#         tokenized_input = self.tokenizer(
#             prompt,
#             padding="max_length",
#             truncation=True,
#             max_length=self.max_length,
#             return_tensors="pt"
#         )

#         return {
#             "input_ids": tokenized_input["input_ids"].squeeze(),
#             "attention_mask": tokenized_input["attention_mask"].squeeze(),
#             "answer": answer,
#         }


In [129]:
train_dataset = train_dataset.drop(columns=["question", "date", "groups"])
test_dataset = test_dataset.drop(columns=["question", "date", "groups"])

In [130]:
train_dataset.head()

,answers,prompt
0,"[['HAIL', 'RAIN', 'SLEET', 'SNOW'], ['BUCKS', ...","{'prompt': [{'role': 'system', 'content': 'You..."
1,"[['BOOT', 'LOAFER', 'PUMP', 'SNEAKER'], ['FOOT...","{'prompt': [{'role': 'system', 'content': 'You..."
2,"[['CHEEK', 'EYE', 'MOUTH', 'NOSE'], ['CHOW', '...","{'prompt': [{'role': 'system', 'content': 'You..."
3,"[['ADIDAS', 'NIKE', 'PUMA', 'REEBOK'], ['CABAR...","{'prompt': [{'role': 'system', 'content': 'You..."
4,"[['HULU', 'NETFLIX', 'PEACOCK', 'PRIME'], ['KE...","{'prompt': [{'role': 'system', 'content': 'You..."


In [131]:
train_dataset["prompt"] = train_dataset["prompt"].apply(lambda x: json.dumps(x)) 
test_dataset["prompt"] = test_dataset["prompt"].apply(lambda x: json.dumps(x))

In [135]:
class CustomGRPOTrainer:
    def __init__(self, model_id="Qwen/Qwen2-0.5B-Instruct", output_dir="GRPO-test"):
        self.model_id = model_id
        self.output_dir = output_dir
        self.model = None
        self.tokenizer = None
        self.trainer = None
        self.training_args = None

    def load_model(self):
        """Loads the base model and applies LoRA fine-tuning."""
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_id)
        self.model = AutoModelForCausalLM.from_pretrained(
            self.model_id,
            torch_dtype="auto",
            device_map="auto",
        )

        lora_config = LoraConfig(
            task_type="CAUSAL_LM",
            r=8,
            lora_alpha=32,
            lora_dropout=0.1,
            target_modules=["q_proj", "v_proj"],
        )

        self.model = get_peft_model(self.model, lora_config)

    def format_reward(self, completions, **kwargs):
        """Reward function that checks if the completion matches your Connections puzzle format."""
        pattern = r"^Answer: \[\[.*?\], \[.*?\], \[.*?\], \[.*?\]\]\s+Group: \[.*?, .*?, .*?, .*?\]\.$"
        matches = [re.match(pattern, completion) for completion in completions]
        return [1.0 if match else 0.0 for match in matches]
    
    def accuracy_reward(self, completions, **kwargs):
        """Reward function that checks if the completion matches the ground truth."""
        answer = kwargs["answers"]
        completion_contents = [completion[0]["content"] for completion in completions]
        rewards = []
        for content, answer in zip(completion_contents, answer):
            gold_parsed = parse(answer, extraction_mode="first_match", extraction_config=[LatexExtractionConfig()])
            answer_parsed = parse(content, extraction_mode="first_match", extraction_config=[LatexExtractionConfig()])
            if len(gold_parsed) != 0:
                try:
                    rewards.append(float(verify(answer_parsed, gold_parsed)))
                except Exception:
                    rewards.append(0.0)
            else:
                rewards.append(1.0)
        return rewards

    def configure_training(self):
        """Configures the training arguments for GRPOTrainer."""
        self.training_args = GRPOConfig(
            output_dir=self.output_dir,
            learning_rate=1e-5,
            gradient_accumulation_steps=16,
            num_train_epochs=1,
            bf16=True,  # Ensure bf16 is supported on GPU
            max_completion_length=64,
            num_generations=4,
            max_prompt_length=128,
            report_to=["tensorboard"],
            logging_steps=10,
            push_to_hub=False,
            save_strategy="steps",
            save_steps=10,
        )

    def train_model(self, train_dataset):
        """Trains the model using GRPOTrainer."""
        if self.model is None or self.training_args is None:
            raise ValueError("Model and training configuration must be set before training.")

        self.trainer = GRPOTrainer(
            model=self.model,
            reward_funcs=[self.format_reward, self.accuracy_reward],
            args=self.training_args,
            train_dataset=train_dataset,
        )

        print("Training started...")
        self.trainer.train()
        print("Training completed!")
    
    def save_trained_model(self):
        """Saves the trained model to the Hugging Face Hub."""
        self.trainer.save_model(self.training_args.output_dir)

In [136]:
# Instantiate and set up the trainer
trainer = CustomGRPOTrainer()
trainer.load_model()
trainer.configure_training()

In [137]:
train_dataset = Dataset.from_pandas(train_dataset)
print(train_dataset)
trainer.train_model(train_dataset)

AttributeError: 'Dataset' object has no attribute 'columns'

In [ ]:
# Save and push trained model
trainer.save_trained_model()

In [ ]:
def load_model(model_id):
    trained_model = AutoModelForCausalLM.from_pretrained(
        model_id,
        torch_dtype="auto",
        device_map="auto",
    )
    trained_tokenizer = AutoTokenizer.from_pretrained(model_id)

    return {"model": trained_model, "tokenizer": trained_tokenizer}

In [ ]:
def generate_with_reasoning(prompt, trained_model, trained_tokenizer):
    # Build the prompt from the dataset
    print(type(prompt))
    prompt = " ".join(entry["content"] for entry in prompt)

    # Tokenize and move to the same device as the model
    inputs = trained_tokenizer(prompt, return_tensors="pt").to(trained_model.device)

    # Generate text without gradients
    start_time = time.time()
    with torch.no_grad():
        output_ids = trained_model.generate(**inputs, max_length=500)
    end_time = time.time()

    # Decode and extract model response
    generated_text = trained_tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Get inference time
    inference_duration = end_time - start_time

    # Get number of generated tokens
    num_input_tokens = inputs["input_ids"].shape[1]
    num_generated_tokens = output_ids.shape[1] - num_input_tokens

    return generated_text, inference_duration, num_generated_tokens

In [ ]:
# Initialize and load the model
trainer = load_model("Qwen/Qwen2-0.5B-Instruct")

# Set the trained model and tokenizer
trained_model = trainer['model']
trained_tokenizer = trainer['tokenizer']

# Define a sample prompt
prompt = test_dataset.iloc[0]["prompt"]

# Generate text with reasoning
response, duration, num_tokens = generate_with_reasoning(prompt, trained_model, trained_tokenizer)

print("Response:", response)
print(f"Inference duration: {duration:.2f} seconds")
print(f"Generated tokens: {num_tokens}")

<class 'dict'>


TypeError: string indices must be integers, not 'str'